In [2]:
using Pkg
Pkg.add("DFTK")
using DFTK

   Resolving package versions...


   Installed LinearMaps ─ v3.10.2


    Updating `~/.julia/environments/v1.9/Project.toml`
  [acf6eb54] + DFTK v0.6.9
    Updating `~/.julia/environments/v1.9/Manifest.toml`


  [ada6cbde] + Bravais v0.1.8
  [23470ee3] + Brillouin v0.5.13
  [b0b7db55] + ComponentArrays v0.13.14
  [acf6eb54] + DFTK v0.6.9
  [6bd331d2] + DftFunctionals v0.2.3
  [c3f9d41a] + DirectQhull v0.2.0
  [d9f16b24] + Functors v0.4.4
  [a9efe35a] + InteratomicPotentials v0.2.6
  [42fd0dbc] + IterativeSolvers v0.9.2
  [66e17ffc] + Libxc v0.3.15
  [7a12625a] + LinearMaps v3.10.2
  [da04e1cc] + MPI v0.20.11
  [3da0fdf6] + MPIPreferences v0.1.8
  [eebad327] + PkgVersion v0.3.2
  [f27b6e38] + Polynomials v3.2.13
⌅ [cb339c56] + PseudoPotentialIO v0.1.1
  [f2b01f46] + Roots v2.0.17
  [f761d5c5] + Spglib v0.6.1
⌅ [4093c41a] + StructHelpers v0.1.5
  [1d63c593] + LLVMOpenMP_jll v15.0.4+0
  [25af9330] + Libxc_GPU_jll v6.1.0+2
  [a56a6d9d] + Libxc_jll v6.1.0+0
  [7cb0a576] + MPICH_jll v4.1.2+0
  [f1f71cc9] + MPItrampoline_jll v5.3.1+0
  [9237b28f] + MicrosoftMPI_jll v10.1.3+4
  [fe0851c0] + OpenMPI_jll v4.1.5+0
⌅ [784f63db] + Qhull_jll v8.0.1003+0
  [ac4a9f1e] + spglib_jll v1.16.5+0
        Info Pac

Precompiling 

...

  ✓ LinearMaps → LinearMapsSparseArraysExt


  ✓ ComponentArrays → ComponentArraysSciMLBaseExt


  ✓ LinearMaps → LinearMapsChainRulesCoreExt


  ✓ LinearMaps → LinearMapsStatisticsExt
  ✓ LinearSolve → LinearSolveIterativeSolversExt


  ✓ DFTK
  10 dependencies successfully precompiled in 49 seconds. 488 already precompiled.


In [6]:
# Define a simple 2D square lattice
a = 10.0  # Lattice constant
lattice = a / 2 * [1  1 0; 
                   1 -1 0; 
                   0  0 2]

3×3 Matrix{Float64}:
 5.0   5.0   0.0
 5.0  -5.0   0.0
 0.0   0.0  10.0

In [7]:
# Define the atomic positions and species
atoms = [ElementPsp(:H, psp=load_psp("hgh/lda/h-q1")) => [ones(3)/8, -ones(3)/8]]

1-element Vector{Pair{ElementPsp, Vector{Vector{Float64}}}}:
 ElementPsp(H, psp="hgh/lda/h-q1") => [[0.125, 0.125, 0.125], [-0.125, -0.125, -0.125]]

In [13]:
# Create the crystal
model = Model(lattice, atoms)

MethodError: MethodError: no method matching Model(::Matrix{Float64}, ::Vector{Pair{ElementPsp, Vector{Vector{Float64}}}})

Closest candidates are:
  Model(::AbstractMatrix{T}) where T<:Real
   @ DFTK ~/.julia/packages/DFTK/k0wtY/src/Model.jl:95
  Model(::AbstractMatrix{T}, !Matched::Vector{<:DFTK.Element}) where T<:Real
   @ DFTK ~/.julia/packages/DFTK/k0wtY/src/Model.jl:95
  Model(::AbstractMatrix{T}, !Matched::Vector{<:DFTK.Element}, !Matched::Vector{<:AbstractVector}; model_name, εF, n_electrons, disable_electrostatics_check, magnetic_moments, terms, temperature, smearing, spin_polarization, symmetries) where T<:Real
   @ DFTK ~/.julia/packages/DFTK/k0wtY/src/Model.jl:95


In [ ]:
using DFTK
using Unitful
using UnitfulAtomic
using LinearAlgebra

# Define the convergence parameters (these should be increased in production)
L = 20  # height of the simulation box
kgrid = [6, 6, 1]
Ecut = 15
temperature = 1e-3

# Define the geometry and pseudopotential
a = 4.66  # lattice constant
a1 = a*[1/2,-sqrt(3)/2, 0]
a2 = a*[1/2, sqrt(3)/2, 0]
a3 = L*[0  , 0        , 1]
lattice = [a1 a2 a3]
C1 = [1/3,-1/3,0.0]  # in reduced coordinates
C2 = -C1
positions = [C1, C2]
C = ElementPsp(:C, psp=load_psp("hgh/pbe/c-q4"))
atoms = [C, C]

# Run SCF
model = model_PBE(lattice, atoms, positions; temperature)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis)